# Mod 7 Final Project

Jeremy Owens

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
import keras
from keras import initializers, regularizers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, advanced_activations
from keras.optimizers import RMSprop, Adam, SGD

from sklearn.preprocessing import StandardScaler

%matplotlib inline
sns.set_style('darkgrid')

Using TensorFlow backend.


In [42]:
industries = ['Agriculture', 'Construction', 'Education_Health', 
              'Finance', 'Leisure_Hospitality', 'Manufacturing', 
              'Mining_Logging', 'Professional', 'Retail', 
              'Trade_Transportation_Utilities'] 

### Industry employement

In [50]:
industry_df = pd.DataFrame()
for industry in industries:
    temp_df = pd.read_csv('Season_no/Employees - {} no season.csv'.format(industry))
    temp_df.columns = ['Date_time', '{}_emp'.format(industry)]
    temp_df['{}_emp'.format(industry)] = temp_df['{}_emp'.format(industry)] * 1000
    temp_df['Date_time'] = pd.to_datetime(temp_df['Date_time'])
    temp_df.set_index('Date_time', inplace=True)
    temp_df = temp_df[temp_df.index.year >= 1948]
    temp_df = temp_df.astype({'{}_emp'.format(industry): 'int64'})
    industry_df = pd.concat([industry_df, temp_df], axis=1)
    
display(industry_df.head())
display(industry_df.tail())
display(industry_df.info())

,Agriculture_emp,Construction_emp,Education_Health_emp,Finance_emp,Leisure_Hospitality_emp,Manufacturing_emp,Mining_Logging_emp,Professional_emp,Retail_emp,Trade_Transportation_Utilities_emp
Date_time,,,,,,,,,,
1948-01-01,6846000,2000000,2017000,1697000,2664000,14402000,1006000,2850000,4431300,9537000
1948-02-01,6587000,1862000,2013000,1708000,2636000,14344000,997000,2843000,4380300,9497000
1948-03-01,6677000,1949000,2031000,1719000,2668000,14373000,1011000,2863000,4431600,9553000
1948-04-01,7222000,2090000,2065000,1733000,2678000,14082000,911000,2872000,4432800,9492000
1948-05-01,7556000,2230000,2088000,1739000,2696000,14019000,1023000,2900000,4456500,9564000


,Agriculture_emp,Construction_emp,Education_Health_emp,Finance_emp,Leisure_Hospitality_emp,Manufacturing_emp,Mining_Logging_emp,Professional_emp,Retail_emp,Trade_Transportation_Utilities_emp
Date_time,,,,,,,,,,
2019-03-01,2243000,7174000,24208000,8590000,16260000,12778000,746000,21067000,15576600,27491000
2019-04-01,2281000,7380000,24308000,8607000,16575000,12778000,747000,21333000,15624900,27558000
2019-05-01,2476000,7543000,24218000,8636000,16961000,12811000,754000,21399000,15691600,27687000
2019-06-01,2422000,7700000,24038000,8714000,17396000,12925000,762000,21577000,15776100,27829000
2019-07-01,2569000,7756000,23951000,8761000,17504000,12935000,762000,21605000,15781300,27794000


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 859 entries, 1948-01-01 to 2019-07-01
Data columns (total 10 columns):
Agriculture_emp                       859 non-null int64
Construction_emp                      859 non-null int64
Education_Health_emp                  859 non-null int64
Finance_emp                           859 non-null int64
Leisure_Hospitality_emp               859 non-null int64
Manufacturing_emp                     859 non-null int64
Mining_Logging_emp                    859 non-null int64
Professional_emp                      859 non-null int64
Retail_emp                            859 non-null int64
Trade_Transportation_Utilities_emp    859 non-null int64
dtypes: int64(10)
memory usage: 73.8 KB


None

In [51]:
emp_levels = ['16-19', '20-24', '25-34', '35-44', '45-54']

In [52]:
emp_levels_df = pd.DataFrame()
for emp in emp_levels:
    temp_df = pd.read_csv('Season_no/Employees - {} no season.csv'.format(emp))
    temp_df.columns = ['Date_time', 'Age_{}_emp'.format(emp)]
    temp_df['Age_{}_emp'.format(emp)] = temp_df['Age_{}_emp'.format(emp)] * 1000
    temp_df['Date_time'] = pd.to_datetime(temp_df['Date_time'])
    temp_df.set_index('Date_time', inplace=True)
    emp_levels_df = pd.concat([emp_levels_df, temp_df], axis=1)
    
display(emp_levels_df.head())
display(emp_levels_df.tail())
display(emp_levels_df.info())

,Age_16-19_emp,Age_20-24_emp,Age_25-34_emp,Age_35-44_emp,Age_45-54_emp
Date_time,,,,,
1948-01-01,3518000,6767000,13420000,12786000,10334000
1948-02-01,3575000,6705000,13436000,12848000,10333000
1948-03-01,3653000,6613000,13473000,12887000,10386000
1948-04-01,3770000,6768000,13671000,12910000,10526000
1948-05-01,3757000,6858000,13757000,12965000,10543000


,Age_16-19_emp,Age_20-24_emp,Age_25-34_emp,Age_35-44_emp,Age_45-54_emp
Date_time,,,,,
2019-03-01,4887000,13925000,35686000,32785000,32279000
2019-04-01,4672000,14114000,35602000,33001000,32287000
2019-05-01,4888000,14288000,35703000,33080000,32150000
2019-06-01,5941000,14699000,35699000,32933000,31816000
2019-07-01,6409000,14787000,35369000,32904000,31638000


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 859 entries, 1948-01-01 to 2019-07-01
Data columns (total 5 columns):
Age_16-19_emp    859 non-null int64
Age_20-24_emp    859 non-null int64
Age_25-34_emp    859 non-null int64
Age_35-44_emp    859 non-null int64
Age_45-54_emp    859 non-null int64
dtypes: int64(5)
memory usage: 40.3 KB


None

In [110]:
age_1947_pop_df = pd.DataFrame()
for year in range(1947, 1980):
    temp_df = pd.read_excel('US_pop/pop_1948-1979/US Pop {}.xls'.format(str(year)), skiprows=8).iloc[:86, 1:2]
    temp_df.columns = ['Pop']
    temp_df['Age_16-19_pop'] = temp_df.iloc[16:20].Pop.sum()
    temp_df['Age_20-24_pop'] = temp_df.iloc[20:25].Pop.sum()
    temp_df['Age_25-34_pop'] = temp_df.iloc[25:35].Pop.sum()
    temp_df['Age_35-44_pop'] = temp_df.iloc[35:45].Pop.sum()
    temp_df['Age_45-54_pop'] = temp_df.iloc[45:55].Pop.sum()
    temp_df['Age_55-64_pop'] = temp_df.iloc[55:65].Pop.sum()
    temp_df = temp_df.drop('Pop', axis=1).iloc[:12]
    datelist = pd.date_range(pd.datetime(year,4,1), periods=12, freq='MS')
    temp_df.set_index(datelist, inplace=True)
    age_1947_pop_df = pd.concat([age_1947_pop_df, temp_df])

age_1947_pop_df = age_1947_pop_df.astype('int64')
display(age_1947_pop_df.info())
display(age_1947_pop_df.head())
display(age_1947_pop_df.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 396 entries, 1947-04-01 to 1980-03-01
Data columns (total 6 columns):
Age_16-19_pop    396 non-null int64
Age_20-24_pop    396 non-null int64
Age_25-34_pop    396 non-null int64
Age_35-44_pop    396 non-null int64
Age_45-54_pop    396 non-null int64
Age_55-64_pop    396 non-null int64
dtypes: int64(6)
memory usage: 21.7 KB


None

,Age_16-19_pop,Age_20-24_pop,Age_25-34_pop,Age_35-44_pop,Age_45-54_pop,Age_55-64_pop
1947-04-01,9096939,11814354,23235875,20421307,16970667,12527372
1947-05-01,9096939,11814354,23235875,20421307,16970667,12527372
1947-06-01,9096939,11814354,23235875,20421307,16970667,12527372
1947-07-01,9096939,11814354,23235875,20421307,16970667,12527372
1947-08-01,9096939,11814354,23235875,20421307,16970667,12527372


,Age_16-19_pop,Age_20-24_pop,Age_25-34_pop,Age_35-44_pop,Age_45-54_pop,Age_55-64_pop
1979-11-01,17447096,20933574,35087548,24406493,23119836,20958964
1979-12-01,17447096,20933574,35087548,24406493,23119836,20958964
1980-01-01,17447096,20933574,35087548,24406493,23119836,20958964
1980-02-01,17447096,20933574,35087548,24406493,23119836,20958964
1980-03-01,17447096,20933574,35087548,24406493,23119836,20958964


In [174]:
age_1990_pop_df = pd.read_csv('US_pop/pop_1990-1999/US Population 1990-2000.csv')
age_1990_pop_df.dropna(inplace=True)
age_1990_pop_df.drop(['Male', 'Female'], axis=1, inplace=True)
age_1990_pop_df = age_1990_pop_df[(age_1990_pop_df.Age != 'All Age') & (age_1990_pop_df.Age != '100+')]
age_1990_pop_df= age_1990_pop_df.astype({'Age': 'int64', 'Pop': 'int64'})
age_1990_pop_df['Date'] = pd.to_datetime(age_1990_pop_df.Date)

display(age_1990_pop_df.info())
display(age_1990_pop_df.head())
display(age_1990_pop_df.tail())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12100 entries, 1 to 12470
Data columns (total 3 columns):
Date    12100 non-null datetime64[ns]
Age     12100 non-null int64
Pop     12100 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 378.1 KB


None

,Date,Age,Pop
1,1990-04-01,0,3947313
2,1990-04-01,1,3769554
3,1990-04-01,2,3702679
4,1990-04-01,3,3641649
5,1990-04-01,4,3703886


,Date,Age,Pop
12466,2000-04-01,95,98095
12467,2000-04-01,96,72680
12468,2000-04-01,97,52844
12469,2000-04-01,98,36003
12470,2000-04-01,99,27162


In [195]:
warnings.filterwarnings('ignore')
dict_age_groups = {'Date': [],
                   'Age_16-19_pop': [],
                   'Age_20-24_pop': [],
                   'Age_25-34_pop': [],
                   'Age_35-44_pop': [],
                   'Age_45-54_pop': [],
                   'Age_55-64_pop': []}
# date = age_1990_pop_df.Date.unique()[0]
for date in age_1990_pop_df.Date.unique():
    dict_age_groups['Date'].append(date)
    dict_age_groups['Age_16-19_pop'].append(age_1990_pop_df[age_1990_pop_df.Date == date]
                                                           [(age_1990_pop_df.Age >= 16) & 
                                                            (age_1990_pop_df.Age < 20)].Pop.sum())
    dict_age_groups['Age_20-24_pop'].append(age_1990_pop_df[age_1990_pop_df.Date == date]
                                                           [(age_1990_pop_df.Age >= 20) & 
                                                            (age_1990_pop_df.Age < 25)].Pop.sum())
    dict_age_groups['Age_25-34_pop'].append(age_1990_pop_df[age_1990_pop_df.Date == date]
                                                           [(age_1990_pop_df.Age >= 25) & 
                                                            (age_1990_pop_df.Age < 35)].Pop.sum())
    dict_age_groups['Age_35-44_pop'].append(age_1990_pop_df[age_1990_pop_df.Date == date]
                                                           [(age_1990_pop_df.Age >= 35) & 
                                                            (age_1990_pop_df.Age < 45)].Pop.sum())
    dict_age_groups['Age_45-54_pop'].append(age_1990_pop_df[age_1990_pop_df.Date == date]
                                                           [(age_1990_pop_df.Age >= 45) & 
                                                            (age_1990_pop_df.Age < 55)].Pop.sum())
    dict_age_groups['Age_55-64_pop'].append(age_1990_pop_df[age_1990_pop_df.Date == date]
                                                           [(age_1990_pop_df.Age >= 55) & 
                                                            (age_1990_pop_df.Age < 65)].Pop.sum())
age_1990_popgroup_df = pd.DataFrame.from_dict(dict_age_groups)
age_1990_popgroup_df.set_index('Date', inplace=True)

# print(dict_to_add)
display(age_1990_popgroup_df.info())
display(age_1990_popgroup_df.head())
display(age_1990_popgroup_df.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 121 entries, 1990-04-01 to 2000-04-01
Data columns (total 6 columns):
Age_16-19_pop    121 non-null int64
Age_20-24_pop    121 non-null int64
Age_25-34_pop    121 non-null int64
Age_35-44_pop    121 non-null int64
Age_45-54_pop    121 non-null int64
Age_55-64_pop    121 non-null int64
dtypes: int64(6)
memory usage: 6.6 KB


None

,Age_16-19_pop,Age_20-24_pop,Age_25-34_pop,Age_35-44_pop,Age_45-54_pop,Age_55-64_pop
Date,,,,,,
1990-04-01,14413349,18613683,42536470,37164280,25028231,21114356
1990-05-01,14376042,18616612,42552331,37270501,25072161,21108575
1990-06-01,14337523,18616157,42565595,37388330,25117998,21103903
1990-07-01,14285636,18614697,42574792,37499153,25172093,21100311
1990-08-01,14227169,18623893,42581903,37623619,25235678,21095956


,Age_16-19_pop,Age_20-24_pop,Age_25-34_pop,Age_35-44_pop,Age_45-54_pop,Age_55-64_pop
Date,,,,,,
1999-12-01,16078948,18407077,39585150,44882073,37192126,24070665
2000-01-01,16095676,18438844,39555097,44886939,37315661,24117741
2000-02-01,16089785,18484688,39522634,44885529,37419687,24171185
2000-03-01,16086054,18540723,39501667,44886153,37525015,24222050
2000-04-01,16091700,18592596,39468491,44886989,37641311,24273352
